In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from scraping_helper_functions import cook_chess_soup
from scraping_helper_functions import get_chess_data

In [3]:
# This is an extremely important cell, very very necessary
def lentil(x):
    return len(x)

In [4]:
# URL = "https://www.chess.com/games/search?opening=&openingId=&p1=Magnus+Carlsen&p2=&mr=&lsty=1&year=&lstMoves=1&moves=&fen=&sort=page=1"
# page = requests.get(URL)

# soup = BeautifulSoup(page._content, "html.parser")

Scrape Chess.com for all of Magnus's otb chess games

In [5]:
# broth = soup.html.body.find('div', class_ = "base-layout").find('div', class_ = "base-container").main
# veggies = broth.find('div', class_ = "layout-column-one").find("div", class_ = "v5-section").find('div', class_ = "v5-section-content-wide").find_all('div', class_ = "post-preview-list-component")[2]
# noodles = veggies.find('div', class_ = 'master-games-table-responsive').table.tbody
# meats = noodles.find_all('tr')

Now we have a list of table entries in html that contains data about each of Magnus's games (on this page at least)
So, lets loop through this list and extract the important data into a dataframe 

Loop through the different pages and grab all the list elements containing data about Magnus's otb games

In [6]:
html_data = []

# Right now this only scrapes games up until 2015 (the first 60 pages)
for num in range(0, 60):
    URL = f"https://www.chess.com/games/search?opening=&openingId=&p1=Magnus%20Carlsen&p2=&mr=&lsty=1&year=&lstMoves=1&moves=&fen=&sort=page%3D3&page={num}"
    html_data.extend(cook_chess_soup(URL))

In [7]:
lentil(html_data)

1500

In [8]:
white_player_list = []
white_player_rating_list = []
black_player_list = []
black_player_rating_list = []
result_list = []
game_length_list = []
year_list = []
opening_name_list = []
opening_list = []

for chunk in html_data:
    data = get_chess_data(chunk)

    white_player_list.append(data[0])
    white_player_rating_list.append(data[1])
    black_player_list.append(data[2])
    black_player_rating_list.append(data[3])
    result_list.append(data[4])
    game_length_list.append(data[5])
    year_list.append(data[6])
    opening_name_list.append(data[7])
    opening_list.append(data[8])

White player rating nan
Black player rating nan
White player rating nan
Black player rating nan
White player rating nan
Black player rating nan
Black player rating nan
Black player rating nan
Black player rating nan
Black player rating nan
Black player rating nan
Black player rating nan
Black player rating nan
Black player rating nan
Black player rating nan
Black player rating nan
Black player rating nan


Now that we have all the lists of data, build the dataframe

In [9]:
games_df = pd.DataFrame()

#There is probably a better way to do this but whatever
games_df['white_player'] = white_player_list
games_df['white_player_rating'] = white_player_rating_list
games_df['black_player'] = black_player_list
games_df['black_player_rating'] = black_player_rating_list
games_df['result'] = result_list
games_df['length'] = game_length_list
games_df['year'] = year_list
games_df['opening_name'] = opening_name_list
games_df['opening_moves'] = opening_list

In [10]:
games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   white_player         1500 non-null   object
 1   white_player_rating  1497 non-null   object
 2   black_player         1500 non-null   object
 3   black_player_rating  1486 non-null   object
 4   result               1500 non-null   object
 5   length               1500 non-null   object
 6   year                 1500 non-null   object
 7   opening_name         1500 non-null   object
 8   opening_moves        1500 non-null   object
dtypes: object(9)
memory usage: 105.6+ KB


Ok, we have the data. Lets clean it.

In [11]:
#Lets do the easy things first: cleaning the ratings, year, result, length, and opening
games_df['result'] = games_df['result'].map(lambda x: x.strip())
games_df['length'] = games_df['length'].map(lambda x: x.strip())
games_df['opening_moves'] = games_df['opening_moves'].map(lambda x: x.strip())
games_df['opening_name'] = games_df['opening_name'].map(lambda x: x.strip())

games_df

,white_player,white_player_rating,black_player,black_player_rating,result,length,year,opening_name,opening_moves
0,Magnus Carlsen,(2881),Wesley So,(2741),½-½,29,2021,"Ruy López Opening: Berlin, Rio Gambit Accepted...",1. e4 e5 2. Nf3 Nc6 3. Bb5 Nf6
1,Levon Aronian,(2761),Magnus Carlsen,(2881),½-½,60,2021,"Ruy López Opening: Morphy Defense, Anti-Marsha...",1. e4 e5 2. Nf3 Nc6 3. Bb5 a6
2,Daniil Dubov,(2710),Magnus Carlsen,(2847),0-1,41,2021,Queen's Gambit Declined: Austrian Variation,1. d4 d5 2. c4 c5 3. Nf3 cxd4
3,Hikaru Nakamura,(2736),Magnus Carlsen,(2847),0-1,39,2021,Queen's Gambit Declined: Janowski Variation,1. d4 d5 2. c4 e6 3. Nc3 a6
4,Andrey Esipenko,(2716),Magnus Carlsen,(2847),½-½,36,2021,"Giuoco Piano Game: Main Line, Giuoco Pianissim...",1. e4 e5 2. Nf3 Nc6 3. Bc4 Bc5
...,...,...,...,...,...,...,...,...,...
1495,Magnus Carlsen,(2876),Veselin Topalov,(2798),0-1,60,2015,Semi-Slav Defense,1. d4 d5 2. c4 c6 3. Nf3 Nf6
1496,Magnus Carlsen,(2862),Ivan Saric,(2666),½-½,49,2015,Sicilian Defense: Old Sicilian Variation,1. e4 c5 2. Nf3 Nc6 3. Nc3 Nf6
1497,Magnus Carlsen,(2853),Maxime Vachier-Lagrave,(2731),1-0,43,2015,"English Opening: Anglo-Indian, King's Indian D...",1. Nf3 Nf6 2. c4 g6 3. g3 Bg7
1498,Magnus Carlsen,(2853),Alexander Grischuk,(2771),0-1,66,2015,"Sicilian Defense: Open, Najdorf, Opočenský Var...",1. e4 c5 2. Nf3 d6 3. d4 cxd4


In [12]:
def rating_to_int(rating):
    if type(rating) != float:
        return int(rating[1:5])
    else:
        return rating

In [13]:
games_df['white_player_rating'] = games_df['white_player_rating'].map(rating_to_int)
games_df['black_player_rating'] = games_df['black_player_rating'].map(rating_to_int)

In [14]:
games_df

,white_player,white_player_rating,black_player,black_player_rating,result,length,year,opening_name,opening_moves
0,Magnus Carlsen,2881.0,Wesley So,2741.0,½-½,29,2021,"Ruy López Opening: Berlin, Rio Gambit Accepted...",1. e4 e5 2. Nf3 Nc6 3. Bb5 Nf6
1,Levon Aronian,2761.0,Magnus Carlsen,2881.0,½-½,60,2021,"Ruy López Opening: Morphy Defense, Anti-Marsha...",1. e4 e5 2. Nf3 Nc6 3. Bb5 a6
2,Daniil Dubov,2710.0,Magnus Carlsen,2847.0,0-1,41,2021,Queen's Gambit Declined: Austrian Variation,1. d4 d5 2. c4 c5 3. Nf3 cxd4
3,Hikaru Nakamura,2736.0,Magnus Carlsen,2847.0,0-1,39,2021,Queen's Gambit Declined: Janowski Variation,1. d4 d5 2. c4 e6 3. Nc3 a6
4,Andrey Esipenko,2716.0,Magnus Carlsen,2847.0,½-½,36,2021,"Giuoco Piano Game: Main Line, Giuoco Pianissim...",1. e4 e5 2. Nf3 Nc6 3. Bc4 Bc5
...,...,...,...,...,...,...,...,...,...
1495,Magnus Carlsen,2876.0,Veselin Topalov,2798.0,0-1,60,2015,Semi-Slav Defense,1. d4 d5 2. c4 c6 3. Nf3 Nf6
1496,Magnus Carlsen,2862.0,Ivan Saric,2666.0,½-½,49,2015,Sicilian Defense: Old Sicilian Variation,1. e4 c5 2. Nf3 Nc6 3. Nc3 Nf6
1497,Magnus Carlsen,2853.0,Maxime Vachier-Lagrave,2731.0,1-0,43,2015,"English Opening: Anglo-Indian, King's Indian D...",1. Nf3 Nf6 2. c4 g6 3. g3 Bg7
1498,Magnus Carlsen,2853.0,Alexander Grischuk,2771.0,0-1,66,2015,"Sicilian Defense: Open, Najdorf, Opočenský Var...",1. e4 c5 2. Nf3 d6 3. d4 cxd4


In [18]:
games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   white_player         1500 non-null   object 
 1   white_player_rating  1497 non-null   float64
 2   black_player         1500 non-null   object 
 3   black_player_rating  1486 non-null   float64
 4   result               1500 non-null   object 
 5   length               1500 non-null   object 
 6   year                 1500 non-null   object 
 7   opening_name         1500 non-null   object 
 8   opening_moves        1500 non-null   object 
dtypes: float64(2), object(7)
memory usage: 105.6+ KB


In [25]:
games_df['length'] = games_df['length'].astype('int64')

In [26]:
games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   white_player         1500 non-null   object 
 1   white_player_rating  1497 non-null   float64
 2   black_player         1500 non-null   object 
 3   black_player_rating  1486 non-null   float64
 4   result               1500 non-null   object 
 5   length               1500 non-null   int64  
 6   year                 1500 non-null   object 
 7   opening_name         1500 non-null   object 
 8   opening_moves        1500 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 105.6+ KB


In [28]:
games_df['result'].value_counts()

½-½    634
1-0    501
0-1    365
Name: result, dtype: int64

In [31]:
games_df['opening_name'].value_counts()

Queen's Gambit Declined: 3.Nf3 Nf6                                                           81
Indian Game                                                                                  33
Ruy López Opening: Berlin Defense, 4.d3 Bc5                                                  26
Giuoco Piano Game: Main Line, Giuoco Pianissimo Variation                                    25
Queen's Pawn Opening: Symmetrical Variation                                                  22
                                                                                             ..
Giuoco Piano Game: Main Line, Giuoco Pianissimo Variation, 5...a6                             1
Ruy López Opening: Morphy Defense, Columbus Variation, 4...Nf6 5.O-O b5 6.Bb3 Bc5 7.c3 d6     1
King's Indian Defense: Orthodox, Exchange Variation, 8.Qxd8 Rxd8 9.Bg5 Re8 10.Nd5             1
Sicilian Defense: Open, Najdorf, English Attack, 6...e5                                       1
Queen's Gambit Declined: Lasker Defense 

In [44]:
opening = []
opening_variant = []
for game in games_df['opening_name'].map(lambda x: x.split(':')):
    opening.append(game[0])
    try: 
        opening_variant.append(game[1])
    except:
        opening_variant.append(np.NaN)

In [48]:
opening

['Ruy López Opening',
 'Ruy López Opening',
 "Queen's Gambit Declined",
 "Queen's Gambit Declined",
 'Giuoco Piano Game',
 'Ruy López Opening',
 'Nimzo-Indian Defense',
 'Ruy López Opening',
 'English Opening',
 "Queen's Gambit Declined",
 'Ruy López Opening',
 'Nimzo-Indian Defense',
 'Sicilian Defense',
 "Queen's Gambit Declined",
 'Ruy López Opening',
 'Sicilian Defense',
 'Ruy López Opening',
 'Catalan Opening',
 'Catalan Opening',
 'Ruy López Opening',
 'English Opening',
 'Ruy López Opening',
 'Giuoco Piano Game',
 'Sicilian Defense',
 'English Opening',
 'Ruy López Opening',
 'Ruy López Opening',
 "Queen's Gambit Declined",
 "Queen's Gambit Declined",
 'Giuoco Piano Game',
 'Ruy López Opening',
 'Nimzo-Indian Defense',
 'Ruy López Opening',
 'English Opening',
 "Queen's Gambit Declined",
 'Ruy López Opening',
 'Nimzo-Indian Defense',
 'Sicilian Defense',
 "Queen's Gambit Declined",
 'Ruy López Opening',
 'Sicilian Defense',
 'Ruy López Opening',
 'Catalan Opening',
 'Catalan Ope

In [46]:
opening_variant

[' Berlin, Rio Gambit Accepted, 5.Re1 Nd6 6.Nxe5',
 ' Morphy Defense, Anti-Marshall Variation, 8...Bb7 9.d3 d6 10.c3',
 ' Austrian Variation',
 ' Janowski Variation',
 ' Main Line, Giuoco Pianissimo Variation',
 ' Morphy Defense, Anti-Marshall Variation, 8...Bb7 9.d3 d6',
 ' Classical, Noa Variation, 5.cxd5 Qxd5 6.e3 c5',
 ' Berlin Defense, 4.d3 Bc5',
 ' Two Knights Variation',
 ' 3.Nf3 Nf6',
 ' Morphy Defense, Anti-Marshall Variation',
 ' Classical, Noa Variation, 5.cxd5 exd5 6.Bg5 h6 7.Bh4 c5',
 ' Canal Attack, 3...Nd7 4.O-O Nf6 5.Re1 a6 6.Bf1',
 ' Three Knights Variation',
 ' Berlin Defense, 4.d3 Bc5',
 ' Canal Attack, 3...Nd7 4.O-O Nf6 5.Re1 a6 6.Bf1',
 ' Morphy Defense, Deferred Steinitz Defense, 6.c3',
 ' Open Defense, Classical Line, 6.O-O O-O',
 ' Closed, 4...Be7 5.Nf3 O-O 6.O-O',
 " Berlin, l'Hermet, Berlin Wall Defense",
 " Anglo-Indian, King's Knight Variation",
 ' Morphy Defense, Breyer, Zaitsev Hybrid Variation, 11.Nbd2 Bb7 12.Bc2 Re8 13.Nf1 Bf8 14.Ng3 g6 15.a4',
 ' Main L

In [49]:
games_df['opening'] = opening

In [50]:
games_df['opening_variant'] = opening_variant

In [55]:
len(games_df['opening'].value_counts())

53